### 이상거래 예측하기 - lightGBM
- XGBoost 보다 빠르고 높은 정확도 보여줌
- 예측의ㅣ 영향을 미친 변수의 중요도도 확인
- 변수 종류가 많고, 데이터가 클수록 상대적으로 더 뛰어남
- 복잡한 모델인만큼 해석의 어려움이 있음
- 하이퍼파라미터 튜닝이 까다로움

- 이미지나 자연어가 아닌 표로 정리된 데이터라면 거의 모든 상황ㅇ에서 활용 가능

In [62]:
import numpy as np
import matplotlib.pyplot as  plt
import  pandas as pd
import seaborn as sns
from sympy.polys.dispersion import dispersionset



In [63]:
filename = 'fraud.csv'
data = pd.read_csv(filename)
data.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 22 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   trans_date_trans_time  object 
 1   cc_num                 int64  
 2   merchant               object 
 3   category               object 
 4   amt                    float64
 5   first                  object 
 6   last                   object 
 7   gender                 object 
 8   street                 object 
 9   city                   object 
 10  state                  object 
 11  zip                    int64  
 12  lat                    float64
 13  long                   float64
 14  city_pop               int64  
 15  job                    object 
 16  dob                    object 
 17  trans_num              object 
 18  unix_time              int64  
 19  merch_lat              float64
 20  merch_long             float64
 21  is_fraud               int64  
dtypes: float64(5), int

In [65]:
#trans_data 어쩌고: 거래시간
# cc_num: 카드번호
# merchant: 거래상점
# category: 거래상점의 범주
# amt: 거래금액
# first/last : 이름
# gender : 성별
#street/city/state/zip : 주소
# lat/long : 고객 주소의 위도/경도
# city_pop : 고객 거주지에 속하는 도시의 인구수
# job : 직업
# dob : 생년월일
#trans_num: 거래번호
# unix_time: 거래시간
# merch_lat : 상점의 위도
# merch_long : 상점의 경도
#is_fraud : 사기거래여부(종속변수)

In [66]:
data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 22 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trans_date_trans_time  1852394 non-null  object 
 1   cc_num                 1852394 non-null  int64  
 2   merchant               1852394 non-null  object 
 3   category               1852394 non-null  object 
 4   amt                    1852394 non-null  float64
 5   first                  1852394 non-null  object 
 6   last                   1852394 non-null  object 
 7   gender                 1852394 non-null  object 
 8   street                 1852394 non-null  object 
 9   city                   1852394 non-null  object 
 10  state                  1852394 non-null  object 
 11  zip                    1852394 non-null  int64  
 12  lat                    1852394 non-null  float64
 13  long                   1852394 non-null  float64
 14  city_pop          

In [67]:
round(data.describe(),2)

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,1.852394e+06,1852394.00,1852394.00,1852394.00,1852394.00,1852394.00,1.852394e+06,1852394.00,1852394.00,1852394.00
mean,4.173860e+17,70.06,48813.26,38.54,-90.23,88643.67,1.358674e+09,38.54,-90.23,0.01
std,1.309115e+18,159.25,26881.85,5.07,13.75,301487.62,1.819508e+07,5.11,13.76,0.07
min,6.041621e+10,1.00,1257.00,20.03,-165.67,23.00,1.325376e+09,19.03,-166.67,0.00
25%,1.800429e+14,9.64,26237.00,34.67,-96.80,741.00,1.343017e+09,34.74,-96.90,0.00
50%,3.521417e+15,47.45,48174.00,39.35,-87.48,2443.00,1.357089e+09,39.37,-87.44,0.00
75%,4.642255e+15,83.10,72042.00,41.94,-80.16,20328.00,1.374581e+09,41.96,-80.25,0.00
max,4.992346e+18,28948.90,99921.00,66.69,-67.95,2906700.00,1.388534e+09,67.51,-66.95,1.00


In [68]:
data.drop(['first','last','street','city','state','zip','trans_num','unix_time','job','merchant'],axis=1,inplace=True)

In [69]:
data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])

In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 12 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   trans_date_trans_time  datetime64[ns]
 1   cc_num                 int64         
 2   category               object        
 3   amt                    float64       
 4   gender                 object        
 5   lat                    float64       
 6   long                   float64       
 7   city_pop               int64         
 8   dob                    object        
 9   merch_lat              float64       
 10  merch_long             float64       
 11  is_fraud               int64         
dtypes: datetime64[ns](1), float64(5), int64(3), object(3)
memory usage: 169.6+ MB


In [71]:
data['amt'] = pd.to_numeric(data['amt'], errors='coerce')

amt_info = (data.groupby('cc_num')['amt'].agg(['mean','std']).reset_index())
amt_info.head()

,cc_num,mean,std
0,60416207185,59.257796,142.869746
1,60422928733,65.483159,92.042844
2,60423098130,96.376084,1000.693872
3,60427851591,107.487550,131.014534
4,60487002085,64.096925,153.207660


In [72]:
data = data.merge(amt_info, on='cc_num', how='left')

In [73]:
# Z_score 계산

data['amt_z_score'] = (data['amt'] - data['mean']) / data['std']

In [74]:
data[['amt','mean','std','amt_z_score']].head()

,amt,mean,std,amt_z_score
0,4.97,89.408743,127.530101,-0.662108
1,107.23,56.078113,159.201852,0.321302
2,220.11,69.924272,116.688602,1.287064
3,45.00,80.090040,280.077880,-0.125287
4,41.96,95.341146,94.322842,-0.565941


In [75]:
data['amt'] = pd.to_numeric(data['amt'], errors='coerce')
#cc_num, category 별로 amt 평균, 표준편차 계산
amt_info = (data.groupby(['cc_num','category'])['amt'].agg(['mean','std']).reset_index())

In [76]:
data['cate_z_score'] = (data['amt'] - data['mean']) / data['std']

In [77]:
#mean, std는 z 점수 구하고 삭제
data.drop(['mean', 'std'], axis=1, inplace=True)


In [82]:
import geopy.distance
import time

In [81]:
data['merch_coord'] = pd.Series(zip(data['merch_lat'], data['merch_long']))
data['cust_coord'] = pd.Series(zip(data['lat'], data['long']))

In [84]:
start_time = time.time()
data['distance'] = data.apply(lambda x: geopy.distance.distance(x['merch_coord'],
                                                                x['cust_coord']).km, axis=1)

end_time = time.time()
print(end_time - start_time)

263.05107951164246


In [85]:
data['distance'] = pd.to_numeric(data['distance'], errors='coerce')

In [86]:
## cc_num 별 거리평균, 거리표준편차 계산
distance_info = (data.groupby('cc_num', as_index=False).agg(mean_distance=('distance','mean'),
                                                            std_distance=('distance','std')))

In [87]:
distance_info.head()

,cc_num,mean_distance,std_distance
0,60416207185,73.534542,28.698149
1,60422928733,78.986422,29.299240
2,60423098130,77.826916,28.187560
3,60427851591,75.711544,28.977432
4,60487002085,79.442907,28.767086


In [88]:
# 데이터 합치기
data = data.merge(distance_info, on='cc_num', how='left')

In [89]:
data['distance_z_score'] = (data['distance'] - data['mean_distance']) / data['std_distance']

In [90]:
data.drop(['mean_distance', 'std_distance'], axis=1, inplace=True)

In [92]:
data.tail()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z_score,cate_z_score,merch_coord,cust_coord,distance,distance_z_score
1852389,2020-12-31 23:59:07,30560609640617,health_fitness,43.77,M,40.4931,-91.8912,519,1966-02-13,39.946837,-91.333331,0,-0.167639,-0.167639,"(39.946837, -91.333331)","(40.4931, -91.8912)",77.032467,0.052081
1852390,2020-12-31 23:59:09,3556613125071656,kids_pets,111.84,M,29.0393,-95.4401,28739,1999-12-27,29.661049,-96.186633,0,0.364619,0.364619,"(29.661049, -96.186633)","(29.0393, -95.4401)",100.023736,0.693848
1852391,2020-12-31 23:59:15,6011724471098086,kids_pets,86.88,F,46.1966,-118.9017,3684,1981-11-29,46.658340,-119.715054,0,-0.015209,-0.015209,"(46.65834, -119.715054)","(46.1966, -118.9017)",80.887812,0.288293
1852392,2020-12-31 23:59:24,4079773899158,travel,7.99,M,44.6255,-116.4493,129,1965-12-15,44.470525,-117.080888,0,-0.592099,-0.592099,"(44.470525, -117.080888)","(44.6255, -116.4493)",53.060882,-0.730061
1852393,2020-12-31 23:59:34,4170689372027579,entertainment,38.13,M,35.6665,-97.4798,116001,1993-05-10,36.210097,-97.036372,0,-0.152588,-0.152588,"(36.210097, -97.036372)","(35.6665, -97.4798)",72.380990,-0.159439


In [93]:
#생년월일 정보를 활용해서 나이 구하기
data['age'] = 2021 - pd.to_datetime(data['dob']).dt.year

In [94]:
data.drop(['cc_num','lat','long','merch_lat','merch_long','dob','merch_coord','cust_coord'], axis=1, inplace=True)

In [95]:
data.head()

,trans_date_trans_time,category,amt,gender,city_pop,is_fraud,amt_z_score,cate_z_score,distance,distance_z_score,age
0,2019-01-01 00:00:18,misc_net,4.97,F,3495,0,-0.662108,-0.662108,78.773821,0.030974,33
1,2019-01-01 00:00:44,grocery_pos,107.23,F,149,0,0.321302,0.321302,30.216618,-1.475224,43
2,2019-01-01 00:00:51,entertainment,220.11,M,4154,0,1.287064,1.287064,108.102912,1.160572,59
3,2019-01-01 00:01:16,gas_transport,45.00,M,1939,0,-0.125287,-0.125287,95.685115,0.818063,54
4,2019-01-01 00:03:06,misc_pos,41.96,M,99,0,-0.565941,-0.565941,77.702395,0.064037,35


In [96]:
data = pd.get_dummies(data, columns=['category','gender'], drop_first=True)

In [97]:
data.head()

,trans_date_trans_time,amt,city_pop,is_fraud,amt_z_score,cate_z_score,distance,distance_z_score,age,category_food_dining,...,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M
0,2019-01-01 00:00:18,4.97,3495,0,-0.662108,-0.662108,78.773821,0.030974,33,False,...,False,False,False,True,False,False,False,False,False,False
1,2019-01-01 00:00:44,107.23,149,0,0.321302,0.321302,30.216618,-1.475224,43,False,...,False,False,False,False,False,False,False,False,False,False
2,2019-01-01 00:00:51,220.11,4154,0,1.287064,1.287064,108.102912,1.160572,59,False,...,False,False,False,False,False,False,False,False,False,True
3,2019-01-01 00:01:16,45.00,1939,0,-0.125287,-0.125287,95.685115,0.818063,54,False,...,False,False,False,False,False,False,False,False,False,True
4,2019-01-01 00:03:06,41.96,99,0,-0.565941,-0.565941,77.702395,0.064037,35,False,...,False,False,False,False,True,False,False,False,False,True


In [98]:
data.set_index('trans_date_trans_time', inplace=True)

In [99]:
data.head()

,amt,city_pop,is_fraud,amt_z_score,cate_z_score,distance,distance_z_score,age,category_food_dining,category_gas_transport,...,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M
trans_date_trans_time,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:18,4.97,3495,0,-0.662108,-0.662108,78.773821,0.030974,33,False,False,...,False,False,False,True,False,False,False,False,False,False
2019-01-01 00:00:44,107.23,149,0,0.321302,0.321302,30.216618,-1.475224,43,False,False,...,False,False,False,False,False,False,False,False,False,False
2019-01-01 00:00:51,220.11,4154,0,1.287064,1.287064,108.102912,1.160572,59,False,False,...,False,False,False,False,False,False,False,False,False,True
2019-01-01 00:01:16,45.00,1939,0,-0.125287,-0.125287,95.685115,0.818063,54,False,True,...,False,False,False,False,False,False,False,False,False,True
2019-01-01 00:03:06,41.96,99,0,-0.565941,-0.565941,77.702395,0.064037,35,False,False,...,False,False,False,False,True,False,False,False,False,True
